# Data Mining and Preprocessing

## Данные опросов населения Нидерландов о мобильности

Данные опросов населения Нидерландов изначально представлены в формате данных SPSS и являются результатами двух социологических исследований -- MON и OViN. При этом исследование OViN является достаточно стабильным по структуре и набору переменных, в то время как MON является переходной формой исследования от OVD (1985-2003) к OViN (2009 - н.в.)
Исходя из изменчивости структуры, а также необходимости перевода нидерландскоязычных данных на русский язык проводилась предварительная подготовка данных. Она залючалась в:

- переводе нидерландскоязычных названий переменных в удобный для последующей обработки вид
- присвоении текстовых значений на русском языке категориальным переменным, кодированным целыми числами в таблице
- приведении имен колонок данных MON и OVIN в единый вид

### Импорт данных и библиотек

In [1]:
import pandas as pd
from pyreadstat import pyreadstat
from collections import defaultdict, OrderedDict
import os
from tqdm import tqdm
import json

In [2]:
os.chdir('D:/bachelors/datasets/')

In [17]:
df.rename(columns = mapper_columns).to_excel('D:/bachelors/data_venes_renamed_columns.xlsx')

In [3]:
path_MON = 'D:/bachelors/datasets/MobNed_dat/'
path_OViN = 'D:/bachelors/datasets/OViN_dat/'

l_dir_MON = os.listdir(path_MON)
l_dir_OViN = os.listdir(path_OViN)


iterator = {
    'MON' : {'path' : path_MON,
             'list_dir' : l_dir_MON},
    'OViN' : {'path' : path_OViN,
              'list_dir' : l_dir_OViN}
}

### Выгрузка метаданных из таблиц SPSS для последющего перевода и отбора значений

In [1]:
def get_variables(meta, variables):
    """
    returns metadataframe extracted from MON or OViN data
    """
    current_meta = pd.DataFrame({
        'variable' : meta.column_names,
        'description' : meta.column_labels
    }).assign(
        variable = lambda x: x['variable'].str.lower(),
        description = lambda x: x['description'].str.lower()
    )
    current_meta = current_meta[~current_meta.variable.isin(variables.variable)]
    print(len(current_meta))
    
    return current_meta

In [5]:
def get_values(variables_dict):
    """
    returns values labels in dutch language to use in further translation
    """
    values = pd.DataFrame()
    for var in tqdm(variables_dict):
        current_var_map = variables_dict[var]
        values = values.append(pd.DataFrame({
            'variable' : [var for i in current_var_map.keys()],
            'values' : list(current_var_map.keys()),
            'meaning' : list(current_var_map.values())
       }))
        values['variable'] = values.variable.str.lower()
        
    return values

In [ ]:
variables_table = OrderedDict()
for data in iterator:
    variables_values = OrderedDict()
    l_dir = iterator[data]['list_dir']
    root_path = iterator[data]['path']
    variables = pd.DataFrame(columns = ['variable', 'description'])
    
    for l in tqdm(l_dir):
        meta = pyreadstat.read_sav(root_path + l)[1]
        variables = variables.append(get_variables(meta, variables))
        
        variables_values[l.split('.')[0]] = get_values(meta.variable_value_labels)
        
    variables_table[data] = variables
    
    writer = pd.ExcelWriter(f'D:/bachelors/datasets/Variables/{data}_labels.xlsx')
    for sheet in variables_values:
        variables_values[sheet].to_excel(writer, sheet_name = sheet, index = False)
    writer.save()
    
    
writer = pd.ExcelWriter('D:/bachelors/datasets/Variables/variables_descriptions.xlsx')
for sheet in variables_table:
    variables_table[sheet].to_excel(writer, sheet_name = sheet, index = False)
writer.save()

### Создание словарей для переименования и отбора колонок

Экспортированные в результате получения метаданных преобразовывались и переводились, в результате чего названия колонок были переведены английский язык для удобства испоользования. После этого началось создание словарей, которые использовались для перевода колонок

In [7]:
variables_MON = pd.read_excel(r"Variables\variables descriptions_edited.xlsx", sheet_name = 'MON').dropna()
variables_OViN = pd.read_excel(r"Variables\variables descriptions_edited.xlsx", sheet_name = 'OViN').dropna()

In [9]:
columns_mapper = {
    'MON' : dict(zip(variables_MON.variable, variables_MON.eng_name)),
    'OViN' : dict(zip(variables_OViN.variable, variables_OViN.eng_name))    
}

with open('D:/bachelors/datasets/Variables/mappers/columns_mappers.json', 'w') as outfile:
    json.dump(columns_mapper, outfile)

In [6]:
# if `columns_mapper` is not loaded
with open ('Variables/mappers/columns_mappers.json') as infile:
    columns_mapper = json.load(infile)

In [34]:
df, meta = pyreadstat.read_sav('OViN_dat/OViN2014_Databestand.sav')
df.columns = [i.lower() for i in df.columns]
df = df.drop(columns = list(set(df.columns).difference(columns_mapper['OViN'].keys())))
df = df.rename(columns = columns_mapper['OViN'])

In [147]:
def filter_values(values_dict, database_source):
    """
    Selects required columns and maps them with convinient to use names
    """
    new_values = OrderedDict()
    for sheet in values_dict:
        print(sheet)
        current_labels = values_dict[sheet]
        new_values[sheet] = current_labels[
            current_labels.variable.isin(
                columns_mapper[database_source].keys()
            )
        ].assign(
            variable = lambda x: x['variable'].map(columns_mapper[database_source])
        )
        
    return new_values

In [148]:
values_MON_filtered = filter_values(
    pd.read_excel('Variables/MON_labels.xlsx', sheet_name = None),
    'MON'
)

values_OViN_filtered = filter_values(
    pd.read_excel('Variables/OViN_labels.xlsx', sheet_name = None),
    'OViN'
)

Databestand MON 2004
Databestand MON 2005
Databestand MON 2006
Databestand MON 2007
Databestand MON 2008
Databestand MON 2009
OViN2010_Databestand
OViN2011_Databestand_revisie
OViN2012_Databestand_revisie
OViN2013_Databestand
OViN2014_Databestand
OViN2015_Databestand
OViN2016_Databestand
OViN2017_Databestand


In [13]:
def write_excel_multi(odict, path):
    """
    Writes an ordered dictionary of dataframes to the multipage excel spreadsheet
    """
    writer = pd.ExcelWriter(path)
    for sheet in odict:
        odict[sheet].to_excel(writer, sheet_name = sheet, index = False)
    writer.save()

In [14]:
write_excel_multi(values_MON_filtered, 'Variables/MON_Variables_values_v2.xlsx')
write_excel_multi(values_OViN_filtered, 'Variables/OViN_Variables_values_v2.xlsx')

## Словари для переименования значений переменных

In [158]:
MON_values_map = pd.read_excel('Variables/mappers/MON_Variables_values_to_mappers.xlsx', sheet_name = None)
OViN_values_map = pd.read_excel('Variables/mappers/OViN_Variables_values_to_mappers_v2.xlsx', sheet_name = None)

In [159]:
values_map = {
    'MON' : defaultdict(dict),
    'OViN' : defaultdict(dict)
}

In [160]:
for sheet in MON_values_map:
    current_mapper = MON_values_map[sheet]
    
    for var in current_mapper.variable.unique().tolist():
        current_variable = current_mapper[current_mapper.variable == var]
        values_map['MON'][sheet][var] = dict(zip(current_variable['values'], current_variable['mapped']))

In [161]:
for sheet in OViN_values_map:
    current_mapper = OViN_values_map[sheet]
    
    for var in current_mapper.variable.unique().tolist():
        current_variable = current_mapper[current_mapper.variable == var]
        values_map['OViN'][sheet][var] = dict(zip(current_variable['values'], current_variable['mapped']))

In [167]:
with open('Variables/mappers/values_ru_map.json', 'w') as outfile:
    json.dump(values_map, outfile)

In [168]:
with open('Variables/mappers/values_ru_map.json') as infile:
    values_map = json.load(infile)

### Перевод данных и их экспорт

После подготовки словарей, был произведен перевод значений переменных внутри таблиц данных и фильтрация колонок, пригодных для использования в качестве статичных годовых слоев с социально-экономическими характеристиками территории. 

In [5]:
import re

In [ ]:
for data in iterator:
    colnames = columns_mapper[data]
    colnames.pop('hsnor', None) # on the fly fixing of mistakes in colnames data (they had two different columns for mopeds)
    colnames.pop('hhsnor', None)
    colnames.pop('psnor', None)
    colnames.pop('opsnor', None)
    valuesMap = values_map[data]
    l_dir = iterator[data]['list_dir']
    root_path = iterator[data]['path']
    for l in tqdm(l_dir):
        df = pyreadstat.read_sav(root_path + l)[0]
        df.columns = [i.lower() for i in df.columns]
        
        true_keys = list(set(df.columns).intersection(set(colnames.keys())))
        mapper = {k : colnames[k] for k in true_keys}
        
        df = df[list(mapper.keys())].rename(columns = mapper).astype(int, errors = 'ignore')
        current_df_values = valuesMap[l.split('.')[0]]

        for col in list(current_df_values.keys()):
            if (col in ['geo_departure', 'geo_arrival']) and (len(current_df_values[col]) < 100):
                df.loc[
                    df[col].isin(current_df_values['geo_commune'].keys()), col
                ] = df.loc[
                    df[col].isin(current_df_values['geo_commune'].keys()), col
                ].astype(int).astype(str).map(current_df_values['geo_commune'])
            else:
                df.loc[
                    df[col].isin(current_df_values[col].keys()), col
                ] = df.loc[
                    df[col].isin(current_df_values[col].keys()), col
                ].astype(int).astype(str).map(current_df_values[col])
        year = re.findall('\d+', l)[0]
        df.to_csv(f'translated_data/data_{year}.csv', index = None)

## Геокодирование коммун

Исходные данные имеют географическую привязку в виде почтовых индексов, коммун, регионов COROP и провинций Нидерландов. Поскольку в качестве операционной ячейки использовались коммуны, то далее производилось их прямое геокодирование (преобразование адресов в географические координаты)

In [4]:
datasets = defaultdict(list)

In [ ]:
for y in range(2004, 2018):
    datasets[str(y)] = pd.read_csv(f'translated_data/data_{y}.csv')

In [180]:
def getCommunes(df):
    """
    Returns the set of unique communes
    """
    list_sets = [set(df.geo_arrival.dropna()), 
                 set(df.geo_departure.dropna()),
                 set(df.geo_commune.dropna())]
    return set().union(*list_sets)

In [181]:
communes_set = set()
for dset in datasets:
    communes_set = communes_set.union(getCommunes(df))
    
communes = pd.DataFrame({'commune' : list(communes_set)})
communes['commune'] = communes.commune.apply(lambda x: x + ', Netherlands')
communes['lat'], communes['lon'] = pd.Series(), pd.Series()

In [182]:
import geocoder
from time import sleep
import numpy as np

In [183]:
def gcode(location):
    """
    Sends request to geocoding server, returns tuple with coordinates
    """
    try:
        g = geocoder.arcgis(location).json
        return g['lat'], g['lng'], g
    except Exception as e:
        print(e)
        return np.NaN, np.NaN, g

In [ ]:
responses = []
for i, row in tqdm(communes.iterrows()):
    location = row['commune']
    lat, lon, response = gcode(location)
    sleep(1)
    
    responses.append([location, response])
    
    communes.loc[communes.commune == location, 'lat'] = lat
    communes.loc[communes.commune == location, 'lon'] = lon
    
pd.DataFrame(responses, columns = ['commune', 'response']).to_csv('communes_geocoding_responses_v3.csv', index = None)

In [ ]:
communes.plot.scatter('lon', 'lat')

In [ ]:
communes[(communes.lat.isna()) | (communes.lon.isna())].shape

In [213]:
coordinates_map = {
    'lats' : dict(zip(communes.commune, communes.lat)),
    'lons' : dict(zip(communes.commune, communes.lon))
}

with open('Variables/mappers/coordinates_map_v2.json', 'w') as outfile:
    json.dump(coordinates_map, outfile)

In [212]:
import geopandas as gpd
from shapely.geometry import Point

In [216]:
geo_points_communes = gpd.GeoDataFrame(communes, 
                                       crs = {'init' : 'epsg:4326'}, 
                                       geometry = [Point(xy) for xy in zip(communes.lon, communes.lat)])

In [ ]:
geo_points_communes.to_file('communes_points.gpkg', driver = 'GPKG')

### Выделение статистики по территориям  на основании данных опроса

Опрос содержит некоторые  характеристики домохозяйств, которые являются полезными для использования в дальнейшем ходе исследования, поэтому было принято решение выделить из файлов отдельные статистические слои

In [185]:
col_subsets = {
    'sex' : 'dem_sex',
    'ageGroups' : 'dem_ageGroup',
    'occupation' : 'econ_occupation',
    'n_working_hours' : 'econ_work',
    'education' : 'social_education',
    'income_groups' : 'econ_income',
    'income_deciles' : 'econ_hh_income_deciles',
    'vehicles' : ['n_cars', 'n_bikes', 'n_motorsycles', 'n_mopeds', 'n_mopeds', 'n_other_vehicles'],
    'travel_motivation' : 'motiv_motivation',
    'trip_duration' : 'time_travel_time'
}

In [186]:
for datatype in col_subsets:
    series_list = []
    for year in datasets:
        current_dataset = datasets[year]
        columns_gr = ['geo_commune']
        
        if datatype == 'income_groups' and int(year) >= 2010:
            subset = 'econ_hh_income'
        elif datatype in ['income_deciles', 'trip_duration'] and int(year) < 2010:
            continue
        else:
            subset = col_subsets[datatype]
        
        
        if type(subset) == list:
            columns_gr = columns_gr + subset
        else:
            columns_gr.append(subset)
        series_list.append(
            current_dataset.groupby(columns_gr)['uid'].nunique()
        )
    pd.DataFrame(series_list).transpose().to_csv(f'stats_extracted/stats_{datatype}.csv')

##  Получение статистики по коммунам

CBS Open Data предоставляет довольно дробную статистику по коммунам Нидерландов вплоть до структуры экономики.

In [3]:
import cbsodata

In [5]:
datasets = {
    '2019' : '84583NED',
    '2018' : '84286NED',
    '2017' : '83765NED',
    '2016' : '83487NED',
    '2015' : '83220NED',
    '2014' : '82931NED',
    '2013' : '82339NED'
}

In [6]:
values_map = defaultdict(dict)
for year in datasets:
    table_id = datasets[year]
    pd.DataFrame(
        cbsodata.get_data(table_id, typed = True)
    ).to_csv(f'territory_bases/database_{year}.csv', sep = ';', index = None)
    
    
    mapping_columns = {}
    list_variables = cbsodata.get_meta(table_id, 'DataProperties')
    
    for var in list_variables:
        mapping_columns[var['Key']] = var['Title']
        
    values_map[year] = mapping_columns

In [7]:
with open('territory_bases/map_columns.json', 'w') as outfile:
    json.dump(values_map, outfile)

In [8]:
with open('territory_bases/map_columns.json') as infile:
    values_map = json.load(infile)

In [9]:
writer = pd.ExcelWriter('territory_bases/map_columns.xlsx')

for year in values_map:
    pd.DataFrame(
        {'raw_name' : pd.Series(list(values_map[year].keys())),
         'name' : pd.Series(list(values_map[year].values()))}
    ).to_excel(writer, sheet_name = year, index = False)
    
writer.save()

# Budgets data

In [4]:
df = pd.DataFrame(cbsodata.get_data('45042NED', typed = True))

HTTPError: Downloading table '45042NED' failed. 404 Client Error: Not Found for url: https://opendata.cbs.nl/ODataFeed/odata/45042NED/?%24format=json